In [59]:
import tweepy
import os
import json
# import sys
import geocoder
import csv
import datetime
import re
import nltk
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections
import string
import numpy as np
import plotly.offline as py
import plotly.graph_objects as go
import networkx as nx
import pickle
import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stop_words.add('rt')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/harman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [60]:
def remove_url(txt):
#     return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())
    return " ".join(re.sub(r"http\S+", "", txt).split())

def remove_stop_words(words_in_tweet):
    tweets_cleaned = [[word for word in tweet_words if not word in stop_words]
                  for tweet_words in words_in_tweet]
    return tweets_cleaned

def get_count_of_keywords(tweet_words):
    count = 0
    for w in tweet_words:
        if w in keywords:
            count = count + 1
    return count

def preprocess_clean_tweets(tweets):
    tweets_lower_case = [tweet.lower().split() for tweet in tweets]
    tweets_words_cleaned = remove_stop_words(tweets_lower_case)
    return tweets_words_cleaned

def analyse_tweets(all_tweets, hashtag):
    tweets_url_cleaned = [remove_url(tweet) for tweet in all_tweets]
    related_hashtags = {}
    for tweet in tweets_url_cleaned:
        tag_list = []
        for i in tweet.split():
            if i.find("#") > -1 and i != hashtag:
                i = i[i.find('#'):]
                if i[1:].find("#") > 0:
                    splits = i.split('#')
                    for sp in splits:
                        if len(sp) > 0:
                            if (sp.rfind('\'') == len(sp) - 1) or (sp.rfind('\"')  == len(sp) - 1) or (sp.rfind('.')  == len(sp) - 1):
                                sp = sp[:-1]
                            tag_list.append("#"+sp)
                else:
                    if (i.rfind('\'') == len(i) - 1) or (i.rfind('\"')  == len(i) - 1) or (i.rfind('.')  == len(i) - 1):
                                i = i[:-1]
                    tag_list.append(i)
        if len(tag_list) > 0:
            for tag in tag_list:
                if hashtag in tag:
                    continue
                if tag in related_hashtags:
                    related_hashtags[tag] += 1
                else:
                    related_hashtags[tag] = 1
    tweet_words = preprocess_clean_tweets(tweets_url_cleaned)
    # Flatten list of words in clean tweets
    tweet_words_flattened = list(itertools.chain(*tweet_words))
    count_keywords = get_count_of_keywords(tweet_words_flattened)
    return count_keywords, related_hashtags

def create_graph(graph, title):
# Custom function to create an edge between node x and node y, with a given text and width
    def make_edge(x, y, text, width):
        return  go.Scatter(x         = x,
                           y         = y,
                           line      = dict(width = width,
                                       color = 'cornflowerblue'),
#                            hoverinfo = 'text',
#                            text      = ([text]),
                           mode      = 'lines')

    edge_trace = []
    for edge in graph.edges(): 
        char_1 = edge[0]
        char_2 = edge[1]
        x0, y0 = pos_[char_1]
        x1, y1 = pos_[char_2]
    #     text   = char_1 + '--' + char_2 + ': ' + str(graph.edges()[edge]['weight'])

        trace  = make_edge([x0, x1, None], [y0, y1, None], text, 
                           width = 0.3*graph.edges()[edge]['weight']**1.75)
        edge_trace.append(trace)

    # Make a node trace
    node_trace = go.Scatter(x         = [],
                            y         = [],
                            text      = [],
                            textposition = "top center",
                            textfont_size = 8,
                            mode      = 'markers+text',
                            hoverinfo = 'none',
                            marker    = dict(color = [],
                                             size  = [],
                                             line  = None))
    # For each node in midsummer, get the position and size and add to the node_trace
    for node in graph.nodes():
        x, y = pos_[node]
        node_trace['x'] += tuple([x])
        node_trace['y'] += tuple([y])
        node_trace['marker']['color'] += tuple(['cornflowerblue'])
        node_trace['marker']['size'] += tuple([5*graph.nodes()[node]['size']])
        node_trace['text'] += tuple(['<b>' + node + '</b>'])


    # Customize layout
    layout = go.Layout(
        paper_bgcolor='rgba(0,0,0,0)', # transparent background
        plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
        xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
        yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
    )
    # Create figure
    fig = go.Figure(layout = layout )
    fig.update_layout(title_text=title, title_x=0.5)
    # Add all edge traces
    for trace in edge_trace:
        fig.add_trace(trace)
    # Add node trace mulitple times for 
    fig.add_trace(node_trace)
    fig.add_trace(node_trace)
    fig.add_trace(node_trace)
    # Remove legend
    fig.update_layout(showlegend = False)
    # Remove tick labels
    fig.update_xaxes(showticklabels = False)
    fig.update_yaxes(showticklabels = False)
    # Show figure
    fig.show()

In [73]:
keywords = ['protest', 'protests', 'protesting', 'uprising', 'insurgence', 'insurrection', 'rebellion', 'revolt', 
            'insurgency', 'distubance', 'riot', 'rioting', 'riots', 'unrest', 'coup', 'treason', 
            'violence', 'demonstration', 'uproar', 'disorder','arrest','criminal','criminals','revolution'
           ]

# API Keys and Tokens
consumer_key = ''
consumer_secret = ''
access_token = ''
access_token_secret = ''

# Authorization and Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Trends for Specific Country
loc = 'India'   
g = geocoder.osm(loc)

# Switch to use Twitter api 
online = False

In [76]:
hashtag_tweet_dict = {}
hashtag_tweet_keyword_count = {}
related_hashtag_dict = {}

if online:
    closest_loc = api.trends_closest(g.lat, g.lng)
    trends = api.trends_place(closest_loc[0]['woeid'])
    current_datetime = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    # writing a JSON file that has the latest trends for that location
    with open("twitter_{}_trend_".format(loc)+current_datetime+".json","w") as wp:
        wp.write(json.dumps(trends, indent=1))

    hashtags_count = len(trends[0]['trends'])

    for i in range(0, hashtags_count):
        hashtag = trends[0]['trends'][i]['name']
        tweet_vol = trends[0]['trends'][i]['tweet_volume']
        if tweet_vol is None:
            continue
        query = hashtag
        all_tweets = []
        for tweet in tweepy.Cursor(api.search,q=query,
                                   lang="en",tweet_mode='extended').items(50):
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:
                text = tweet.full_text
            all_tweets.append(text)
        hashtag_tweet_dict[hashtag] = all_tweets
    f = open("tweets_"+loc+"_"+current_datetime+'.pkl',"wb")
    pickle.dump(hashtag_tweet_dict,f)
    f.close()
else:
    trend_file = 'twitter_India_trend_20210411-194605.json'
    with open(trend_file) as f:
          trends = json.load(f)
    tweets_file = open("tweets_India_20210411-194605.pkl", "rb")
    hashtag_tweet_dict = pickle.load(tweets_file)

for tag in hashtag_tweet_dict.keys():
    tweets = hashtag_tweet_dict[tag]
    keyword_count, related_hashtags = analyse_tweets(tweets, tag)
    if keyword_count > 0:
        hashtag_tweet_keyword_count[tag] = keyword_count
        related_hashtag_dict[tag] = related_hashtags
print(hashtag_tweet_keyword_count)
print(related_hashtag_dict)

{'#DelhiPolice_TorturingFarmers': 6, 'Dainik Bhaskar Defaming SSR': 1, 'Terrorists': 1}
{'#DelhiPolice_TorturingFarmers': {'#FarmersProtest': 9, '#Farmers_vs_BikauModi': 1, '#DoYouKnow': 1}, 'Dainik Bhaskar Defaming SSR': {'#ClubhouseJournos': 1, '#JusticeForSushantSinghRajput': 1, '#JusticeForDishaSalian': 1}, 'Terrorists': {'#Apr12Coup': 20, '#WhatsHappeningInMyanmar': 19, '#SaveBalochRefugees': 1, '#Mandalay': 1, '#Myanmar': 8, '#WhatsHappeninglnMyanmar': 6, '#MilkTeaAlliance': 8, '#MyayPhyu': 1, '#Taunggyi': 1, '#Apr11Coup': 4, '#Tamu,': 3, '#FlashStrikeMyanmar': 1, '#Turkey': 1, '#TurkeyIsATerrorState': 1, '#UkraineRussia': 1, '#UkraineWar': 1, '#Donbass': 1, '#BreakingNews': 1, '#Parents': 1, '#terrorists': 1, '#encounter:': 1, '#FreeRealityWinner': 1, '#GOPSedition': 1, '#TrumpTreason': 1, '#BlueLivesMatter': 1, '#LockThemAllUp': 1, '#jessicawatkins': 1, '#rileywilliams': 1, '#ashlibabbit': 1, '#locktrumpup': 1, '#TheSaraCarterShow': 1}}


In [85]:
if len(hashtag_tweet_keyword_count.keys()) == 0:
    print("No Data Found")
else:
    print("Continue making graphs")

Continue making graphs
49


In [86]:
##MAKING GRAPH OF TRENDING HASHTAGS OF COUNTRY
trending_hashtags = {}
hashtags_count = len(trends[0]['trends']) 
for i in range(0, hashtags_count):
    if trends[0]['trends'][i]['tweet_volume'] is not None:
        trending_hashtags[trends[0]['trends'][i]['name']] = trends[0]['trends'][i]['tweet_volume']

trending_hashtags = sorted(trending_hashtags.items(), key = lambda kv:(kv[1], kv[0]))

G = nx.Graph()
G.add_node('India', size=10)
count=1
max_size = 5
factor = max_size/len(trending_hashtags)
for trend in trending_hashtags:
    G.add_node(trend[0], size=count*factor+1)
    G.add_edge('India', trend[0], weight = count*factor)
    count = count + 1

nodes = list(G.nodes(data=True))
edges = list(G.edges(data=True))
np.random.shuffle(nodes)

graph=nx.Graph()
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)
pos_ = nx.circular_layout(graph, scale = 1)
pos_['India'] =  np.array([0, 0])
title = 'TRENDING HASHTAGS'
create_graph(graph, title)

In [88]:
## MAKING GRAPH FOR RELATED HASHTAGS OF HASHTAG WITH MOST UNREST
sorted_keyword_dict = sorted(hashtag_tweet_keyword_count.items(), key = lambda kv:(kv[1], kv[0]), reverse=True)

hashtag = sorted_keyword_dict[0][0]
related_hashtags = related_hashtag_dict[hashtag]
related_hashtags = sorted(related_hashtags.items(), key = lambda kv:(kv[1], kv[0]))
G = nx.Graph()
G.add_node(hashtag, size=10)
count=1
max_size = 5
factor = max_size/len(related_hashtags)
for tag in related_hashtags:
    G.add_node(tag[0], size=count*factor+1)
    G.add_edge(hashtag, tag[0], weight = count*factor)
    count = count + 1

nodes = list(G.nodes(data=True))
edges = list(G.edges(data=True))
np.random.shuffle(nodes)

graph=nx.Graph()
graph.add_nodes_from(nodes)
graph.add_edges_from(edges)
pos_ = nx.circular_layout(graph, scale = 1)
pos_[hashtag] =  np.array([0, 0])
title = 'RELATED HASHTAGS'
create_graph(graph, title)